### generate_synthetic_principles.ipynb
This file will take an individual agents value system, find the decision they would take, and then assign them an ethical principle that fits with their view, or is the opposite of their view according to some probability $p$

In [105]:
import pandas as pd
import numpy as np

# Globals
against_scheme = list(np.arange(1.8, 3.1, 0.1))
for_scheme = list(np.arange(1.0, 1.7, 0.1)) + list(np.arange(3.2, 10.0, 0.1))
against_scheme = [round(num, 1) for num in against_scheme]
for_scheme = [round(num, 1) for num in for_scheme]

# Read in data
agent_csv_file = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/processed_data_ess.csv"
df = pd.read_csv(agent_csv_file)
df = df.drop(columns=["a_adp_rel", "a_adp_nonrel"])
df['rel-nonrel'] = (df['rel']) / (df['rel'] + df['nonrel'])
df['nonrel-rel'] = (df['nonrel']) / (df['rel'] + df['nonrel'])
# sanity check
print(df.head())

  country   rel  nonrel  a_div_rel  a_div_nonrel  rel-nonrel  nonrel-rel
0      AT   642    1216   0.047767      0.112390    0.345533    0.654467
1      BE   621    1116  -0.063875     -0.054361    0.357513    0.642487
2      CH   767     666   0.150804      0.262262    0.535241    0.464759
3      CZ   577    1483   0.021375      0.028995    0.280097    0.719903
4      DE  1264    1488   0.002110      0.068100    0.459302    0.540698


In [106]:
def binary_principles(df):
    df['decision'] = np.nan  # Initialize the 'decision' column

    for index, row in df.iterrows():
        df.at[index, 'decision'] = (row['rel-nonrel'] * row['a_div_rel']) + (row['nonrel-rel'] * row['a_div_nonrel'])
        if df.at[index, 'decision'] < 0:
            df.at[index, 'egal'] = 1
            df.at[index, 'util'] = 0
        else:
            df.at[index, 'egal'] = 0
            df.at[index, 'util'] = 1
    return df

def range_principles(df):
    df['decision'] = np.nan  # Initialize the 'decision' column #TODO!!!! 
    for index, row in df.iterrows():
        # note: against_scheme = 1.8 to 3.1
                # for_scheme = 1.0 to 1.7 and 3.2 to 10.0
        total_interviewees = row['rel'] + row['nonrel']

        df.at[index, 'decision'] = (row['rel-nonrel'] * row['a_div_rel']) + (row['nonrel-rel'] * row['a_div_nonrel'])
        if df.at[index, 'decision'] < 0:
            factor = np.random.choice(against_scheme)
            egal = (factor / 10) * total_interviewees
            util = total_interviewees - egal
            egal = round(egal, 0)
            util = round(util, 0)
            # Factor is 1.0 then [egal, util] = [0, 1]
            # Factor is 10.0 then [egal, util] = [1, 0]
            # Factor is 1.8 then [egal, util] = [0.18, 0.82]

            df.at[index, 'egal'] = egal
            df.at[index, 'util'] = util
        else:
            factor = np.random.choice(for_scheme)
            egal = (factor / 10) * total_interviewees
            util = total_interviewees - egal
            egal = round(egal, 0)
            util = round(util, 0)
            df.at[index, 'egal'] = egal
            df.at[index, 'util'] = util
    return df


In [107]:
df = range_principles(df)
df

,country,rel,nonrel,a_div_rel,a_div_nonrel,rel-nonrel,nonrel-rel,decision,egal,util
0,AT,642,1216,0.047767,0.112390,0.345533,0.654467,0.090061,1784.0,74.0
1,BE,621,1116,-0.063875,-0.054361,0.357513,0.642487,-0.057762,330.0,1407.0
2,CH,767,666,0.150804,0.262262,0.535241,0.464759,0.202605,1204.0,229.0
3,CZ,577,1483,0.021375,0.028995,0.280097,0.719903,0.026861,721.0,1339.0
4,DE,1264,1488,0.002110,0.068100,0.459302,0.540698,0.037791,1954.0,798.0
5,EE,802,1118,0.072319,0.049493,0.417708,0.582292,0.059028,710.0,1210.0
6,ES,567,1143,0.014697,0.056285,0.331579,0.668421,0.042495,1659.0,51.0
7,FI,739,1099,-0.098782,-0.005763,0.402067,0.597933,-0.043163,386.0,1452.0
8,FR,1063,899,0.021010,0.112347,0.541794,0.458206,0.062861,844.0,1118.0
9,GB,541,1287,0.008010,0.050505,0.295952,0.704048,0.037929,1280.0,548.0


In [108]:
# Save the principles a file
principles_df = df[['country', 'egal', 'util', 'a_div_rel', 'a_div_nonrel' ]]
principles_df.rename(columns={'egal': 'rel', 'util': 'nonrel'}, inplace=True)

principles_df.to_csv("/home/ia23938/Documents/GitHub/ValueSystemsAggregation/process_data/processed_data_with_principles_ess.csv", index=False)

/tmp/ipykernel_387519/3507457046.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  principles_df.rename(columns={'egal': 'rel', 'util': 'nonrel'}, inplace=True)
